## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [18]:
import os
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from pyproj import CRS
import requests
import geojson

# read in input files
txt_file = "activity_locations.txt"

txt_df = pd.read_csv(txt_file, sep=";")

txt_df.head()

,id,name,addr
0,0,kooirassaarentie,"Koirasaarentie 35, 00590 Helsinki"
1,1,gustaf,"Gustaf Hällströmin katu 2, 00560 Helsinki"


In [20]:
# Geocode activity locations
geocoded = geocode(txt_df['addr'], provider='nominatim', user_agent='autogis_jg', timeout=4)

# Reproject to EPSG 3879
geocoded = geocoded.to_crs(epsg=3879)

geocoded = geocoded.join(txt_df)

geocoded.head()


,geometry,address,id,name,addr
0,POINT (25501265.300 6672841.614),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",0,kooirassaarentie,"Koirasaarentie 35, 00590 Helsinki"
1,POINT (25497942.152 6676901.898),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ...",1,gustaf,"Gustaf Hällströmin katu 2, 00560 Helsinki"


In [21]:
# Check coordinate reference system information (re-project/define if needed)
shopping_centers_gdf = gpd.read_file("shopping_centers.shp")

# Drop columns
shopping_centers_gdf = shopping_centers_gdf.drop(['address'], axis=1)

# Rename column
shopping_centers_gdf = shopping_centers_gdf.rename(columns={'name':'shop_name'})

shopping_centers_gdf.head()

,id,shop_name,addr,geometry
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland",POINT (25.083 60.212)
1,1,Forum,"Mannerheimintie 14-20, 00100 Helsinki, Finland",POINT (24.938 60.169)
2,2,Iso-Omena,"Piispansilta 11, 02230 Espoo, Finland",POINT (24.740 60.160)
3,3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland",POINT (24.814 60.219)
4,4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa",POINT (24.963 60.292)


In [22]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
from shapely.ops import nearest_points

def get_nearest_values(row, other_gdf=shopping_centers_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["geometry"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["geometry"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

geocoded["nearest_shop"] = geocoded.apply(get_nearest_values, other_gdf=shopping_centers_gdf, point_column="geometry",value_column='shop_name', axis=1)

geocoded.head()

,geometry,address,id,name,addr,nearest_shop
0,POINT (25501265.300 6672841.614),"Koirasaarentie, Hopealaakso, Kruunuvuorenranta...",0,kooirassaarentie,"Koirasaarentie 35, 00590 Helsinki",Itis
1,POINT (25497942.152 6676901.898),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ...",1,gustaf,"Gustaf Hällströmin katu 2, 00560 Helsinki",Itis


Can you think of other application cases for the nearest neighbour analysis?

(<shapely.geometry.point.Point object at 0x7f8931a40040>, <shapely.geometry.point.Point object at 0x7f8931a405e0>)
